# Ring-Down Analysis Example

This notebook demonstrates how to use the new OOP API for analyzing real ring-down measurement data.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

from ringdownanalysis import RingDownAnalyzer, RingDownDataLoader


In [ ]:
# Apply consistent plotting style
from ringdownanalysis import plots
plots.apply_plotting_style()



## Initialize Analyzer

The analyzer can be configured with custom estimators if needed.


In [ ]:
# Create analyzer with default estimators
analyzer = RingDownAnalyzer()

# Or customize estimators:
# from ringdownanalysis import NLSFrequencyEstimator, DFTFrequencyEstimator
# nls_est = NLSFrequencyEstimator(tau_known=None)
# dft_est = DFTFrequencyEstimator(window='kaiser', kaiser_beta=9.0)
# analyzer = RingDownAnalyzer(nls_estimator=nls_est, dft_estimator=dft_est)


## Find Data Files


In [ ]:
data_dir = Path('../data')
csv_files = sorted(glob.glob(str(data_dir / '*.csv')))
mat_files = sorted(glob.glob(str(data_dir / '*.mat')))

print(f"Found {len(csv_files)} CSV files and {len(mat_files)} MAT files")


## Analyze Each File

The analyzer performs the complete pipeline:
1. Load data
2. Estimate tau from full data
3. Crop data to 3*tau
4. Estimate frequency using NLS and DFT
5. Estimate noise parameters
6. Calculate CRLB


In [ ]:
results = []

for filepath in csv_files + mat_files:
    try:
        print(f"\nProcessing {Path(filepath).name}...")
        r = analyzer.analyze_file(filepath)
        results.append(r)
        
        print(f"  Sampling frequency: {r['fs']:.2f} Hz")
        print(f"  Estimated tau: {r['tau_est']:.2f} s")
        print(f"  NLS frequency: {r['f_nls']:.6f} Hz")
        print(f"  DFT frequency: {r['f_dft']:.6f} Hz")
        print(f"  CRLB std: {r['crlb_std_f']:.6e} Hz")
    except Exception as e:
        print(f"  Error: {e}")


## Visualize Results


In [ ]:
if len(results) > 0:
    # Plot first result
    r = results[0]
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Time series
    ax = axes[0]
    # Downsample for plotting
    step = max(1, len(r['t']) // 50000)
    ax.plot(r['t'][::step], r['data'][::step], 'b-', alpha=0.5, label='Original')
    step_crop = max(1, len(r['t_crop']) // 50000)
    ax.plot(r['t_crop'][::step_crop], r['data_cropped'][::step_crop], 'r-', alpha=0.7, label='Cropped')
    ax.axvline(3.0 * r['tau_est'], color='g', linestyle='--', label=f"3×τ = {3.0*r['tau_est']:.1f} s")
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Phase (cycles)')
    ax.set_title(f"{r['filename']}")
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Frequency comparison
    ax = axes[1]
    f_nls_all = [r['f_nls'] for r in results]
    f_dft_all = [r['f_dft'] for r in results]
    ax.scatter(f_nls_all, f_dft_all, s=100, alpha=0.6, edgecolors='black')
    f_min = min(min(f_nls_all), min(f_dft_all))
    f_max = max(max(f_nls_all), max(f_dft_all))
    ax.plot([f_min, f_max], [f_min, f_max], 'r--', label='Perfect agreement')
    ax.set_xlabel('NLS Frequency (Hz)')
    ax.set_ylabel('DFT Frequency (Hz)')
    ax.set_title('Frequency Estimation Comparison')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## Summary Statistics


In [ ]:
if len(results) > 0:
    f_nls_all = np.array([r['f_nls'] for r in results])
    f_dft_all = np.array([r['f_dft'] for r in results])
    
    print("Summary Statistics:")
    print(f"  Number of files: {len(results)}")
    print(f"  NLS mean: {np.mean(f_nls_all):.9f} Hz")
    print(f"  NLS std: {np.std(f_nls_all):.6e} Hz")
    print(f"  DFT mean: {np.mean(f_dft_all):.9f} Hz")
    print(f"  DFT std: {np.std(f_dft_all):.6e} Hz")
    print(f"  Mean difference: {np.mean(np.abs(f_nls_all - f_dft_all)):.6e} Hz")
